## OME-Zarr notebook

This notebook will demonstrate the `qim3d` functionalities that specifically target the OME-Zarr file format.

OME-Zarr is a cloud-native, multi-dimensional file format optimized for storing and analyzing large-scale volumetric imaging data. It leverages the Zarr storage framework, which organizes data in a chunked, hierarchical structure, enabling efficient random access to specific regions without loading entire datasets into memory. The format supports multi-resolution storage, allowing visualization and analysis at different levels of detail, improving performance for large datasets. OME-Zarr stores metadata in JSON format following the Open Microscopy Environment (OME) model, ensuring interoperability across a wide range of bioimaging tools. Additionally, its compatibility with modern data science libraries like Dask and Xarray allows for scalable parallel processing, making it an ideal choice for applications in microscopy, medical imaging, and machine learning.

First we will fetch a large data file, which we can save to the OME-Zarr file type. Here we will use the `io.Downloader` class. First we define the variable, and then we evaluate what data we would like to download, by visualizing the options with the 'help' command.

In [1]:
import qim3d

downloader = qim3d.io.Downloader()

help(downloader)

Help on Downloader in module qim3d.io._downloader object:

class Downloader(builtins.object)
 |  Class for downloading large data files available on the [QIM data repository](https://data.qim.dk/).
 |  
 |  Attributes:
 |      folder_name (str or os.PathLike): Folder class with the name of the folder in <https://data.qim.dk/>
 |  
 |  Syntax for downloading and loading a file is `qim3d.io.Downloader().{folder_name}.{file_name}(load_file=True)`
 |  
 |  ??? info "Overview of available data"
 |      Below is a table of the available folders and files on the [QIM data repository](https://data.qim.dk/).
 |  
 |      Folder name         | File name                                                                                                          | File size
 |      ------------------- | ------------------------------------------------------------------------------------------------------------------ | ---------------------------------------------
 |      `Coal`              | `CoalBri

After deciding to use the okinawa crab volume, we fetch the data. We use the 'load_file' argument to load it while downloading it. Additionally, we can use the vizualisation tool `slicer_orthogonal` too explore the volume from three different axes.

In [9]:
vol = downloader.Crab.OkinawaCrab(load_file=True)

File already downloaded:
/home/s214735/qim3d/docs/notebooks/Crab/OkinawaCrab.tif

Loading OkinawaCrab.tif


Loading:   0%|          | 0.00B/1.86GB  [00:00<?, ?B/s]

Loaded shape: (995, 1014, 992)
Using virtual stack


Attempting to visualize the volume in a three-dimensional space will require a lot of computing power due to the size of the volume. However using the OME-Zarr data type, we can visualize it in chunks. Additionally we can save it in the .zarr format.

First we export the file using the `export_ome_zarr` method. We set the downsample rate to 3, such that we have different scales of the volume.

In [3]:
qim3d.io.export_ome_zarr(
    'crab.zarr',
    vol,
    downsample_rate=3,
    replace=True)

Exporting data to OME-Zarr format at crab.zarr
Number of scales: 3
Calculating the multi-scale pyramid
- Scale 0: (995, 1014, 992)
- Scale 1: (332, 338, 331)
- Scale 2: (111, 113, 110)
Writing data to disk


Saving:   0%|          | 0.00/1.08k [00:00<?, ?Chunks/s]


All done!


We can then use the `chunks` method to visualize the chunks from the volume. Here we have both options for exploring the volume slices or the entire 3D object. Additionally we can change the view dimension and the scale of the volume.

In [4]:
qim3d.viz.chunks('crab.zarr')

If we wish to continue working on a downsampled version of the volume, we can load it back in using `import_ome_zarr`. Here we use the `scale` argument to decide what scale we with to import. By writing 'lowest', we can get the lowest resolution, and by writing 0 or 'highest' we can get the highest resolution. In this case we want the volume at scale 1 along with the highest resolution:

In [5]:
vol_lowscale = qim3d.io.import_ome_zarr('crab.zarr', scale=1)
vol_highscale = qim3d.io.import_ome_zarr('crab.zarr', scale='highest')

/home/s214735/miniconda3/envs/qim3d-env/lib/python3.11/site-packages/zarr/creation.py:614: UserWarning: ignoring keyword argument 'read_only'
  compressor, fill_value = _kwargs_compat(compressor, fill_value, kwargs)
Data contains 3 scales:
- Scale 0: (995, 1014, 992)
- Scale 1: (332, 338, 331)
- Scale 2: (111, 113, 110)

Loading scale 1 with shape (332, 338, 331)
Data contains 3 scales:
- Scale 0: (995, 1014, 992)
- Scale 1: (332, 338, 331)
- Scale 2: (111, 113, 110)

Loading scale 0 with shape (995, 1014, 992)


Now we can visually inspect the volume at different scales. In this case, we can separate the first 256 voxels:

In [6]:
vol_low = vol_lowscale[0:256, 0:256, 0:256]
vol_high = vol_highscale[256:512, 256:512, 256:512]

Then we can visualize the volume along the three primary axes.

In [7]:
qim3d.viz.slicer_orthogonal(vol_low)

In [8]:
qim3d.viz.slicer_orthogonal(vol_high)